In [0]:
%sql
SELECT * FROM workspace.expense_analytics.expenses;
CREATE DATABASE IF NOT EXISTS expense_analytics_bronze;
CREATE DATABASE IF NOT EXISTS expense_analytics_silver;
CREATE DATABASE IF NOT EXISTS expense_analytics_gold;

CREATE TABLE IF NOT EXISTS expense_analytics_bronze.expenses_raw
AS 
SELECT * FROM expense_analytics.expenses


In [0]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable

In [0]:
spark = SparkSession.builder.appName("ingestion").getOrCreate()

new_df = spark.read.table("workspace.google_drive.expenses")

target = DeltaTable.forName(spark, "workspace.expense_analytics_bronze.expenses_raw")

target.alias("t").merge(new_df.alias("s"),
                        "t.date = s.date AND t.category = s.category AND t.description = s.description AND t.amount = s.amount AND t.payment_mode = s.payment_mode" ).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()